<a href="https://colab.research.google.com/github/EMEHUAY/repositorio-paralela/blob/main/Guia_2_Taxonomia_de_Flynn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

El siguiente código va a permitir que todo código ejecutado en el colab pueda ser medido

In [6]:
!pip install ipython-autotime

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.64 s


In [8]:
print(sum(range(10)))

45
time: 1.73 ms


Pregunta #1: Que porción de 1 segundo es el valor impreso?

milisegundo



---



A seguir, tenemos una librería de Python llamado **numba** que realiza paralelización automatica. Asi, se puede verificar que al usar prange() se tiene mejor tiempo de ejecución que al usar range()

In [9]:
from numba import njit, prange
import numpy as np

A = np.arange(5, 14000000)
@njit(parallel=True)
def prange_test(A):
    s = 0
    # Without "parallel=True" in the jit-decorator
    # the prange statement is equivalent to range
    for i in prange(A.shape[0]):
        s += A[i]
    return s

print(prange_test(A))

97999992999990
time: 894 ms


In [15]:
from numba import njit, prange

A = np.arange(2, 2)
#@njit(parallel=True)
def prange_test(A):
    s = 0
    # Without "parallel=True" in the jit-decorator
    # the prange statement is equivalent to range
    for i in range(A.shape[0]):
        s += A[i]
    return s

print(prange_test(A))



0
time: 8.2 ms


Pregunta #2: identifique otros valores en A, de manera que, serializando, tengamos mejor resultado que paralelizando



---



La Taxonomia de Flynn define 4 tipos de arquitecturas para computación paralela: SISD, SIMD, MISD, y MIMD. 


---


Pregunta #3 : El ultimo código ejecutado es de tipo?

sisd



---



Pregunta #4: el siguiente código paralelo es de tipo? Comentar el código para justificar su respuesta

In [16]:
import threading
import time

def print_time(name, n):
    count = 0   
    print("Para el Hilo: %s, en el momento: %s, su valor de count es: %s" % ( name, time.ctime(), count))
    while count < 5:
        time.sleep(n)
        count+=1
        print("%s: %s. count %s" % ( name, time.ctime(), count))

        
t1 = threading.Thread(target=print_time, args=("Thread-1", 2, ) )
t2 = threading.Thread(target=print_time, args=("Thread-2", 0, ) )

t1.start()
t2.start()

Para el Hilo: Thread-1, en el momento: Wed Dec  9 15:33:46 2020, su valor de count es: 0
Para el Hilo: Thread-2, en el momento: Wed Dec  9 15:33:46 2020, su valor de count es: 0
Thread-2: Wed Dec  9 15:33:46 2020. count 1
Thread-2: Wed Dec  9 15:33:46 2020. count 2
time: 16.9 ms
Thread-2: Wed Dec  9 15:33:46 2020. count 3
Thread-2: Wed Dec  9 15:33:46 2020. count 4
Thread-2: Wed Dec  9 15:33:46 2020. count 5


SIMD   la instruccion es print_time



---



Una computadora paralela tipo MIMD es utilizado más en la computación distribuida, ejm. Clusters. El siguiente código en python desktop muestra tal funcionamiento

In [17]:
#greeting-server.py
import Pyro4

@Pyro4.expose
class GreetingMaker(object):
    def get_fortune(self, name):
        return "Hello, {0}. Here is your fortune message:\n" \
               "Behold the warranty -- the bold print giveth and the fine print taketh away.".format(name)

daemon = Pyro4.Daemon()                # make a Pyro daemon
uri = daemon.register(GreetingMaker)   # register the greeting maker as a Pyro object

print("Ready. Object uri =", uri)      # print the uri so we can use it in the client later
daemon.requestLoop()                   # start the event loop of the server to wait for calls

ModuleNotFoundError: ignored

time: 18.6 ms


In [ ]:
#greeting-client.py
import Pyro4

uri = input("What is the Pyro uri of the greeting object? ").strip()
name = input("What is your name? ").strip()

greeting_maker = Pyro4.Proxy(uri)         # get a Pyro proxy to the greeting object
print(greeting_maker.get_fortune(name))   # call method normally

Pregunta #5: Explique que hace este código de tipo MIMD



---



Ejercicio Propuesto: Crear un ejemplo que muestre una computación paralela de tipo MISD

**Referencias**

https://wiki.python.org/moin/ParallelProcessing

https://numba.readthedocs.io/en/stable/user/parallel.html

https://ao.gl/how-to-measure-execution-time-in-google-colab/

http://noisymime.org/blogimages/SIMD.pdf